# CAP06-Working with Different Types of Data

In [0]:
val df = spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/datasets/retail-data/by-day/2010-12-01.csv")
  
  df.printSchema()

  df.createOrReplaceTempView("dfTable")

In [0]:
spark.sql("SELECT * FROM dfTable LIMIT 5").show()

## Converting to Spark Types

In [0]:
import org.apache.spark.sql.functions.lit

df.select(lit(5), lit("five"), lit(5.0)).show(5)

## Working with Booleans

### Example 1

In [0]:
import org.apache.spark.sql.functions.col

df.where(col("InvoiceNo").equalTo(536365))
    .select("InvoiceNo", "Description")
    .show(5, false)

### Example 2

In [0]:
df.where(col("InvoiceNo") === 536365)
    .select("InvoiceNo", "Description")
    .show(5, false)

### Example 3

In [0]:
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")
df.where(col("StockCode").isin("DOT")).where(priceFilter.or(descripFilter)).show()

### Example 4

In [0]:
val DOTCodeFilter = col("StockCode") === "DOT"
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")

df.withColumn("isExpensive", DOTCodeFilter.and(priceFilter.or(descripFilter)))
    .where("isExpensive")
    .select("UnitPrice", "isExpensive")
    .show(5)

### Example 5

In [0]:
import org.apache.spark.sql.functions.{expr, not, col}

df.withColumn("isExpensive", not(col("UnitPrice").leq(250)))
    .filter("isExpensive")
    .select("Description", "UnitPrice")
    .show(5)

## Working with Numbers

### Example 1

In [0]:
import org.apache.spark.sql.functions.{expr, pow, round}

val fabricatedQuantity = round(pow(col("Quantity") * col("UnitPrice"), 2) + 5, 2)
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(5)

### Example 2

In [0]:
df.selectExpr(
    "CustomerId",
    "ROUND((POWER((Quantity * UnitPrice), 2.0) + 5), 2) AS realQuantity"
).show(5)

### Example 3

In [0]:
spark.sql("SELECT CustomerId, ROUND(POWER((Quantity * UnitPrice), 2.0) + 5, 2) AS realQuantity FROM dfTable").show(5)

### Example 4

In [0]:
import org.apache.spark.sql.functions.{round, bround}
df.select(round(col("UnitPrice"), 1).alias("rounded"), col("UnitPrice")).show(5)

### Example 5

In [0]:
import org.apache.spark.sql.functions.lit

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

In [0]:
spark.sql("SELECT ROUND(2.5), BROUND(2.5)").show(2)

### Example 6

In [0]:
import org.apache.spark.sql.functions.{corr}

df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

### Example 7

In [0]:
spark.sql("SELECT corr(Quantity, UnitPrice) FROM dfTable").show()

### Example 8

In [0]:
df.describe().show()

### Example 9

In [0]:
val colName = "UnitPrice"
val quantileProbs = Array(0.5)
val relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError) // 2.51

### Example 10

In [0]:
df.stat.crosstab("StockCode", "Country").show(5)

### Example 11

In [0]:
df.stat.freqItems(Seq("StockCode", "Quantity")).show()

### Example 12

In [0]:
import org.apache.spark.sql.functions.monotonically_increasing_id

df.select(monotonically_increasing_id()).show(10)

## Working with Strings

### Example 1

In [0]:
import org.apache.spark.sql.functions.{initcap}

df.select(initcap(col("Description"))).show(2, false)

### Example 2

In [0]:
import org.apache.spark.sql.functions.{lower, upper}

df.select(col("Description"),
    lower(col("Description")),
    upper(col("Description"))
).show(5)

### Example 3

In [0]:
import org.apache.spark.sql.functions.{lit, ltrim, rtrim, rpad, lpad, trim}

df.select(
    ltrim(lit("     HELLO     ")).as("ltrim"),
    rtrim(lit("     HELLO     ")).as("rtrim"),
    trim(lit("     HELLO     ")).as("trim"),
    lpad(lit("HELLO"), 3, " ").as("lp"),
    rpad(lit("HELLO"), 10, " ").as("rp"),
).show(2)

## Regular Expressions

### Example 1

In [0]:
import org.apache.spark.sql.functions.regexp_replace

val simpleColors = Seq("black", "white", "red", "green", "blue")
val regexString = simpleColors.map(_.toUpperCase).mkString("|")
// the | signifies 'OR' in regular expressions syntax

df.select(
    regexp_replace(col("Description"), regexString, "COLOR").alias("color_clean"),
    col("Description")
).show(5)

### Example 2

In [0]:
import org.apache.spark.sql.functions.translate

df.select(
    translate(col("Description"), "LEET", "1337"),
    col("Description")
).show(5)

### Example 3

In [0]:
import org.apache.spark.sql.functions.regexp_extract

val regexString = simpleColors.map(_.toUpperCase).mkString("(", "|", ")")
// the | signifies 'OR' in regular expressions syntax

df.select(
    regexp_extract(col("Description"), regexString, 1).alias("color_clean"),
    col("Description")
).show(5)

### Example 4

In [0]:
val containsBlack = col("Description").contains("BLACK")
val containsWhite = col("Description").contains("WHITE")

df.withColumn("hasSimpleColor", containsBlack.or(containsWhite))
    .where("hasSimpleColor")
    .select("Description")
    .show(5, false)

### Example 5

In [0]:
val simpleColors = Seq("black", "white", "red", "green", "blue")
val selectedColumns = simpleColors.map(color => {
    col("Description").contains(color.toUpperCase).alias(s"is_$color")
}):+expr("*") // could also append this value

df.select(selectedColumns:_*).where(col("is_white").or(col("is_red")))
  .select("Description").show(5, false)

## Working with Dates and Timestamps

### Example 1

In [0]:
import org.apache.spark.sql.functions.{current_date, current_timestamp}

val dateDF = spark.range(10)
    .withColumn("today", current_date())
    .withColumn("now", current_timestamp())

  dateDF.createOrReplaceTempView("dateTable")

  dateDF.printSchema()

### Example 2

In [0]:
import org.apache.spark.sql.functions.{date_sub, date_add}

dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

### Example 3

In [0]:
import org.apache.spark.sql.functions.{datediff, months_between, to_date}

dateDF.withColumn("week_ago", date_sub(col("today"), 7))
      .select(datediff(col("week_ago"), col("today")))
      .show(1)

dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end")
).select(months_between(col("start"), col("end"))).show(1)

### Example 4

In [0]:
import org.apache.spark.sql.functions.{to_date, lit}
spark.range(5)
    .withColumn("date", lit("2017-01-01"))
    .select(to_date(col("date"))).show(1)

### Example 5

In [0]:
import org.apache.spark.sql.functions.to_date

val dateFormat = "yyyy-MM-dd"
val cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date1"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2")
)

cleanDateDF.createOrReplaceTempView("cleanDateTable")

spark.sql("""
    SELECT
          to_date(date1, 'yyyy-MM-dd') AS date1,
          to_date(date2, 'yyyy-MM-dd') AS date2,
          to_date(date1)
    FROM cleanDateTable
""").show()

### Example 6

In [0]:
import org.apache.spark.sql.functions.to_timestamp

cleanDateDF.select(to_timestamp(col("date1"), dateFormat).alias("dateF")).show()

## Coalesce

In [0]:
import org.apache.spark.sql.functions.coalesce

df.select(coalesce(col("Description"), col("CustomerId")).alias("descriptionCustomer")).show()

## Working with Complex Type

### Example 1 - Structs

In [0]:
import org.apache.spark.sql.functions.struct

val complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

complexDF.select("complex.Description")
complexDF.select(col("complex").getField("Description"))
complexDF.select("complex.*")

### Example 2 - Array Split

In [0]:
import org.apache.spark.sql.functions.split

df.select(split(col("Description"), " ").alias("NewColumn")).show(5)

In [0]:
df.select(split(col("Description"), " ").alias("array_col")).selectExpr("array_col[0]").show(5)

### Example 3 - Array Length

In [0]:
import org.apache.spark.sql.functions.size

df.select(size(split(col("Description"), " ")).alias("sizeColumn")).show(5)

### Example 4 - Array Contains

In [0]:
import org.apache.spark.sql.functions.array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE").alias("columnArray")).show(5)

### Example 5 - Explode

In [0]:
import org.apache.spark.sql.functions.{split, explode}

df.withColumn("splitted", split(col("Description"), " "))
    .withColumn("exploded", explode(col("splitted")))
    .select("Description", "InvoiceNo", "exploded").show(5)

### Example 6 - Maps

In [0]:
import org.apache.spark.sql.functions.map

df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(5)

In [0]:
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(5)

## Working with JSON

### Example 1

In [0]:
import org.apache.spark.sql.functions.{get_json_object, json_tuple}

val jsonDF = spark.range(1).selectExpr(""" '{"myJSONKey" : {"myJSONValue" : [1,2,3]}})' as jsonString""")

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column",
    json_tuple(col("jsonString"), "myJSONKey")
).show(2)

In [0]:
jsonDF.selectExpr(
    "get_json_object(jsonString, '$.myJSONKey.myJSONValue[1]') as column",
    "json_tuple(jsonString, 'myJSONKey')"
).show(2)

### Example 2

In [0]:
import org.apache.spark.sql.functions.to_json

df.selectExpr("(InvoiceNo, Description) as myStruct")
  .select(to_json(col("myStruct")))

### Example 3

In [0]:
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.types._

val parsedSchema = StructType(Array(
    new StructField("InvoiceNo", StringType, true),
    new StructField("Description", StringType, true)
))

df.selectExpr("(InvoiceNo, Description) AS myStruct")
    .select(to_json(col("myStruct")).alias("newJSON"))
    .select(from_json(col("newJSON"), parsedSchema), col("newJSON")).show(2)

# End